# BreakItDown Tasks

### 1. Mark video as watched
- If ID exist
- query result skip over id and check again if ID exists
- repeat until not 

### 2. Revised percentage calculation (relative instead of 100%)
Will cause a tag to be dominant tho (weight 1000 while others still 10). Mitigated with intentional interruptions (#4). 

### 3. User class schema
- self
- interest_weights (dict)
- watched_videos (dict)

### 4. Intentional Interruptions
Handle dominations
- 50% -> Suggest relevant interests 
- 75% -> "Have you been enjoying {tag name} so far? "yes". "recommend more". 

### 5. Modify whisper prompt to return most relevant tag first then extract that for on_user_swipe()
in [github](https://github.com/edgurinc/edgur/blob/main/backend/accounts/videoprocessors.py)
- To help us later in handling which tag to add weights to given videos have multiple tags. 

### 6. Decay other interests whenever watching a video
except current video tag 

### 7. Handle irrelevant tags
Acknowledge that it will be filled soon, such as "food" category

### 8. "Cache" next 4 videos
Run uniformDist+reco only on 4th video 

### 9. Min selection Max selection for categories
Like tiktok's

In [3]:
from pprint import pprint
import random

In [14]:
# Constants

new_video_weight = 5
starting_weights = 5

queued_videos_count = 5 # Number of vids to queue before running query again (to save costs) AKA n_results = 5

natural_decay_rate = 0.2

# Simulation Only (replace user input)
chosen_tag = "Microwave"


In [54]:
def create_user_interest_dict(*interests):
    interest_count = len(interests)
    interest_weight = starting_weights
    user_interests = {interest: interest_weight for interest in interests}

    # Calculate total weight
    total_weight = sum(user_interests.values())

    # Calculate relative weight for each interest and round to 2 decimal places
    relative_weights = {interest: round((weight / total_weight) * 100, 2) for interest, weight in user_interests.items()}

    return user_interests, relative_weights

def calculate_relative_weights(user_interests, total_weight):
    relative_weights = {interest: round((weight / total_weight) * 100, 2) for interest, weight in user_interests.items()}
    return relative_weights

In [46]:
# New user selects interests (max 5)
# 10 total - 5 start to discover other 5 naturally

carl_interests = create_user_interest_dict(
    "DevOps",
    "Tests (SAT)",
    "Machine Learning",
    "Cats",
    "Cooking",
    "LEL"
)

("Raw weights: {'DevOps': 1, 'Tests (SAT)': 1, 'Machine Learning': 1, 'Cats': "
 "1, 'Cooking': 1, 'LEL': 1}")
("Relative weights: {'DevOps': 16.67, 'Tests (SAT)': 16.67, 'Machine "
 "Learning': 16.67, 'Cats': 16.67, 'Cooking': 16.67, 'LEL': 16.67}. Total "
 'weight: 100.02000000000001')


In [47]:
# HELPER FUNCTIONS for update_weight


def observe_action_taken(video_tag, liked, shared, watched, loop_count):
    """
    This function calculates the total points based on the user's actions.

    Parameters:
    video_tag (str): The tag of the video.
    liked (bool): Whether the user liked the video.
    shared (bool): Whether the user shared the video.
    watched (bool): Whether the user watched the video.
    loop_count (int): The number of times the user looped the video.

    Returns:
    float: The total points calculated based on the user's actions.
    """
    # Actions and their corresponding points to add
    Actions = {
        "Like": 0.1,
        "Share": 0.2,
        "Watch": 0.1,  # more than 50% of total duration
        # todo: experiment with increasing watch value
        "Loop": 0.05,
    }

    total_points = 0

    if liked:
        total_points += Actions["Like"]
    if shared:
        total_points += Actions["Share"]
    if watched:
        total_points += Actions["Watch"]
    total_points += loop_count * Actions["Loop"]

    print(f"\nUpdating weights for video tag: {video_tag}")
    print(f"Total points to add: {total_points}")

    return total_points


def handle_update_weight(user_dict_interest, video_tag, total_points):
    """
    This function updates the user's interest score based on the total points.

    Parameters:
    user_dict_interest (dict): The user's interest dictionary.
    video_tag (str): The tag of the video.
    total_points (float): The total points calculated based on the user's actions.

    Returns:
    dict: The updated user's interest dictionary.
    """
    # Increase the weight of the interest based on the action points
    user_dict_interest[video_tag] += total_points
    print(f"Updated {video_tag} to {user_dict_interest[video_tag]}\n")

    for interest in user_dict_interest:
        if interest != "Random" and interest != video_tag:
            deduction = natural_decay_rate
            user_dict_interest[interest] -= deduction

            print(
                f"Deducted {deduction} from {interest}, new weight: {user_dict_interest[interest]}"
            )

    return user_dict_interest


def handle_add_new_tag(user_dict_interest, video_tag):
    """
    This function adds a new interest to the user's interest dictionary.

    Parameters:
    user_dict_interest (dict): The user's interest dictionary.
    video_tag (str): The tag of the video.

    Returns:
    dict: The updated user's interest dictionary with the new interest added.
    """
    # Add new interest if it doesn't exist and give initial points
    initial_points = 5
    user_dict_interest[video_tag] = initial_points
    print(f"Added new interest {video_tag} with initial weight: {initial_points}")

    # Deduct initial points proportionally from existing interests (excluding 'Random' and current video_tag)
    total_deductable_weight = sum(
        weight
        for interest, weight in user_dict_interest.items()
        if interest != "Random" and interest != video_tag
    )

    print(
        f"Total weight to deduct from other interests: {initial_points}\nTotal deductable weight (excluding 'Random' and {video_tag}): {total_deductable_weight}\n"
    )

    for interest in user_dict_interest:
        if interest != "Random" and interest != video_tag:
            deduction = (initial_points / total_deductable_weight) * user_dict_interest[
                interest
            ]
            user_dict_interest[interest] -= deduction

            pprint(
                f"Deducted {deduction} from {interest}, new weight: {user_dict_interest[interest]}"
            )

    return user_dict_interest

In [55]:
def watch_video_and_update_weight(
    user_dict_interest,
    video_tag,
    liked=False,
    shared=False,
    watched=False,
    loop_count=0,
):

    total_points = observe_action_taken(video_tag, liked, shared, watched, loop_count)

    if video_tag in user_dict_interest:
        handle_update_weight(user_dict_interest, video_tag, total_points)
    else:
        handle_add_new_tag(user_dict_interest, video_tag)

    # Sort the interests ("key=item[1]") by weight in descending order and keep only the top 10
    user_dict_interest = dict(
        sorted(user_dict_interest.items(), key=lambda item: item[1], reverse=True)[:10]
    )

    # Check total weight to ensure it sums to 100 (for debugging purposes)
    total_weight = sum(user_dict_interest.values())
    print(f"Total weight after update: {total_weight} (should be 100)\n")
    # pprint(f"Updated interests: {user_dict_interest}\n")

    return user_dict_interest

In [49]:
# HELPER FUNCTIONS for update_weight


def observe_action_taken(video_tag, liked, shared, watched, loop_count):
    """
    This function calculates the total points based on the user's actions.

    Parameters:
    video_tag (str): The tag of the video.
    liked (bool): Whether the user liked the video.
    shared (bool): Whether the user shared the video.
    watched (bool): Whether the user watched the video.
    loop_count (int): The number of times the user looped the video.

    Returns:
    float: The total points calculated based on the user's actions.
    """
    # Actions and their corresponding points to add
    Actions = {
        "Like": 0.1,
        "Share": 0.2,
        "Watch": 0.1,  # more than 50% of total duration
        # todo: experiment with increasing watch value
        "Loop": 0.05,
    }

    total_points = 0

    if liked:
        total_points += Actions["Like"]
    if shared:
        total_points += Actions["Share"]
    if watched:
        total_points += Actions["Watch"]
    total_points += loop_count * Actions["Loop"]

    print(f"\nUpdating weights for video tag: {video_tag}")
    print(f"Total points to add: {total_points}")

    return total_points


def handle_update_weight(user_dict_interest, video_tag, total_points):
    """
    This function updates the user's interest score based on the total points.

    Parameters:
    user_dict_interest (dict): The user's interest dictionary.
    video_tag (str): The tag of the video.
    total_points (float): The total points calculated based on the user's actions.

    Returns:
    dict: The updated user's interest dictionary.
    """
    # Increase the weight of the interest based on the action points
    user_dict_interest[video_tag] += total_points
    print(f"Updated {video_tag} to {user_dict_interest[video_tag]}\n")

    # Deduct the total points proportionally from existing interests (excluding 'Random' and the video_tag itself)
    total_deductable_weight = sum(
        weight
        for interest, weight in user_dict_interest.items()
        if interest != "Random" and interest != video_tag
    )

    print(
        f"Total weight to deduct from other interests: {total_points}\nTotal deductable weight (excluding 'Random' and {video_tag}): {total_deductable_weight}"
    )

    for interest in user_dict_interest:
        if interest != "Random" and interest != video_tag:
            deduction = (total_points / total_deductable_weight) * user_dict_interest[
                interest
            ]
            user_dict_interest[interest] -= deduction

            print(
                f"Deducted {deduction} from {interest}, new weight: {user_dict_interest[interest]}"
            )

    return user_dict_interest


def handle_add_new_tag(user_dict_interest, video_tag):
    """
    This function adds a new interest to the user's interest dictionary.

    Parameters:
    user_dict_interest (dict): The user's interest dictionary.
    video_tag (str): The tag of the video.

    Returns:
    dict: The updated user's interest dictionary with the new interest added.
    """
    # Add new interest if it doesn't exist and give initial points
    user_dict_interest[video_tag] = starting_weights
    print(f"Added new interest {video_tag} with initial weight: {starting_weights}")

    # Deduct initial points proportionally from existing interests (excluding 'Random' and current video_tag)
    total_deductable_weight = sum(
        weight
        for interest, weight in user_dict_interest.items()
        if interest != "Random" and interest != video_tag
    )

    print(
        f"Total weight to deduct from other interests: {starting_weights}\nTotal deductable weight (excluding 'Random' and {video_tag}): {total_deductable_weight}\n"
    )

    for interest in user_dict_interest:
        if interest != "Random" and interest != video_tag:
            deduction = (starting_weights / total_deductable_weight) * user_dict_interest[
                interest
            ]
            user_dict_interest[interest] -= deduction

            pprint(
                f"Deducted {deduction} from {interest}, new weight: {user_dict_interest[interest]}"
            )

    return user_dict_interest

In [56]:
def watch_video_and_update_weight(
    user_dict_interest,
    video_tag,
    liked=False,
    shared=False,
    watched=False,
    loop_count=0,
):

    total_points = observe_action_taken(video_tag, liked, shared, watched, loop_count)

    if video_tag in user_dict_interest:
        handle_update_weight(user_dict_interest, video_tag, total_points)
    else:
        handle_add_new_tag(user_dict_interest, video_tag)

    # Sort the interests ("key=item[1]") by weight in descending order and display only the top 10
    user_dict_interest = dict(
        sorted(user_dict_interest.items(), key=lambda item: item[1], reverse=True)[:10]
    )

    # Check total weight to ensure it sums to 100 (for debugging purposes)
    total_weight = sum(user_dict_interest.values())

    # Assuming user_interests is defined and contains the raw weights
    relative_weights = calculate_relative_weights(user_dict_interest, total_weight)

    # Calculate the total of the relative weights
    total_relative_weight = sum(relative_weights.values())

    print(f"Total relative weight after update: {total_relative_weight} (should be 100)\n")
    print(f"Total weight after update: {total_weight} (should be 100)\n")
    # pprint(f"Updated interests: {user_dict_interest}\n")

    return user_dict_interest

In [58]:
# Simulate user actions
# pprint(f"Before update: {carl_interests}")

watch_video_and_update_weight(
    carl_interests, "Cats", liked=True, watched=True, shared=True, loop_count=10
)

# pprint(f"After update: {carl_interests}")


Updating weights for video tag: Cats
Total points to add: 0.9
Updated Cats to 4.6

Total weight to deduct from other interests: 0.9
Total deductable weight (excluding 'Random' and Cats): 2.3000000000000007
Deducted 0.18000000000000002 from DevOps, new weight: 0.28000000000000014
Deducted 0.18000000000000002 from Tests (SAT), new weight: 0.28000000000000014
Deducted 0.18000000000000002 from Machine Learning, new weight: 0.28000000000000014
Deducted 0.18000000000000002 from Cooking, new weight: 0.28000000000000014
Deducted 0.18000000000000002 from LEL, new weight: 0.28000000000000014
Total relative weight after update: 100.02000000000001 (should be 100)

Total weight after update: 6.000000000000001 (should be 100)



{'Cats': 4.6,
 'DevOps': 0.28000000000000014,
 'Tests (SAT)': 0.28000000000000014,
 'Machine Learning': 0.28000000000000014,
 'Cooking': 0.28000000000000014,
 'LEL': 0.28000000000000014}